# Extrapolate Future Prices Using LSTM

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
data = pd.read_csv('ethereum_2015-08-07_2024-09-08.csv')

# Extract the 'Close' price or the target variable (assuming you want to predict 'Close')
prices = data['Close'].values

# Normalize the prices using MinMaxScaler (scaling between 0 and 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(prices.reshape(-1, 1))

# Prepare data for LSTM
# Use the last 60 days as input to predict the next day
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(prices_scaled)):
    X.append(prices_scaled[i-sequence_length:i, 0])
    y.append(prices_scaled[i, 0])

X, y = np.array(X), np.array(y)

# Reshape input for LSTM (samples, time steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))  # Regularisation to prevent overfitting
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=64)

# Prepare the input for the next 90 days prediction
# Use the last 60 days (up to 2024-09-08) as input
last_60_days = prices_scaled[-60:]
X_future = last_60_days.reshape(1, last_60_days.shape[0], 1)

# Predict the next 90 days
future_predictions = []
n_future = 90  # Number of days to predict

for day in range(n_future):
    predicted_price = model.predict(X_future)
    future_predictions.append(predicted_price[0, 0])
    
    # Update X_future to include the new predicted price and drop the oldest one
    X_future = np.append(X_future[:, 1:, :], [[predicted_price]], axis=1)

# Inverse transform the predicted prices to get actual values
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Output the future predictions (next 90 days)
print(future_predictions)
